In [109]:
import requests
import logging
import re
import dateutil.parser
from datetime import datetime
from urllib.parse import urlencode, parse_qs
from functools import wraps
import time


def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper



class TascConnector:
    
    mission_conf = {
        'gaia' : { 
            'cmd_acro': 'gai',
            'job_acro': 'g',
            'object': 'Gaia'
        }
    }
    
    common_params = {
     'queryType': ['run'],
     'ops': ['ops_Routine'],
     'scenarioId': ['NEW_ID'],
     'inputCase': ['DEFAULT_case'],
     'opsConfig': ['opsSelect'],
     'jobConfig': ['jobTable'],
     'scenarioId': ['NEW_ID'],
     'inputCase': ['DEFAULT_case']
    }
    
    
    @staticmethod
    def format_time_filename(time):
        if isinstance(time, str):
            dt = dateutil.parser.parse(time)
        elif isinstance(time, datetime):
            dt = time
        return dt.strftime('%Y%m%dT%H%M%S')
    
    @staticmethod
    def get_filename(category, mission, initial_time, final_time):
        return (f'{mission}_{category}_'
               f'{TascConnector.format_time_filename(initial_time)}_{TascConnector.format_time_filename(final_time)}_'
               f'{TascConnector.format_time_filename(datetime.now())}.txt')
    
    @staticmethod
    def format_time(time):
        if isinstance(time, str):
            dt = dateutil.parser.parse(time)
        elif isinstance(time, datetime):
            dt = time
        return dt.strftime('%Y/%m/%d %H:%M:%S.%f')[:-3]
            
    
    def __init__(self, url='https://www.fd-tasc.info'):
        self.base_url = url
        
    def get_attitude(self, mission, initial_time, final_time, time_step):
        
        config = self.mission_conf.get(mission)
        params = self.common_params
        
        self.__set_mission_params(params, config, 'quat')
        self.__set_time_params(params, initial_time, final_time, time_step)
        self.__set_frame_params(params)
        
        url = self.__build_cmd_url(config.get("cmd_acro"))
        content = self.__get_url(url, params)
        filename = self.get_filename(mission, 'quat', initial_time, final_time)
        self.__extract_pre(content, filename)

    def get_state_vector(self, mission, initial_time, final_time, time_step):
        
        config = self.mission_conf.get(mission)
        params = self.common_params
        
        self.__set_mission_params(params, config, 'state')
        self.__set_time_params(params, initial_time, final_time, time_step)
        self.__set_frame_params(params)
        self.__set_object_params(params, config.get("object"))
        
        url = self.__build_cmd_url(config.get("cmd_acro"))
        content = self.__get_url(url, params)
        filename = self.get_filename(mission, 'state', initial_time, final_time)
        self.__extract_pre(content, filename)
    
    def __build_cmd_url(self, acronym):
        return f'{self.base_url}/{acronym}cmd-cgi-bin/seqgenExec.pl'
    
    @timeit
    def __get_url(self, url, params):
        response = requests.get(url, params)
        if response.status_code == 200:
            return response.content.decode('utf-8')
        else:
            raise ValueError(f'{response.status_code}')
    
    def __extract_pre(self, txt, filename):
        pre_data = re.findall("<pre>(.*?)</pre>", txt, re.DOTALL)
        if len(pre_data) < 1:
            raise ValueError('Cannot extract the <pre/> data')
        
        pre_data = pre_data[0]
        if filename:
            with open(filename, 'w+') as fd:
                fd.write(pre_data)
        else:
            print(pre_data)
    
    def __set_mission_params(self, params, config, operation):
        params['job'] = f'{config.get("job_acro")}_job_tmpl_fdtool_crttab_{operation}.dat'

    def __set_time_params(self, params, initial_time, final_time, time_step, time_scale='UTC'):
        params['TIME_SCALE'] = time_scale,
        params['INITIAL_TIME'] = self.format_time(initial_time)
        params['FINAL_TIME'] = self.format_time(final_time)
        params['TIME_STEP'] = time_step
        
    def __set_frame_params(self, params, frame='mean equatorial J2000'):
        params['FRAME'] = frame
        
    def __set_object_params(self, params, object_name, ref_object='Solar System Barycentre', ltc_flag='NO' ):
        params['OBJECT'] = object_name
        params['REF_OBJECT'] = ref_object
        params['LTC_FLAG'] = ltc_flag
        

In [112]:
tc = TascConnector()
ts = '000 00:01:00.000'
start_time = '2023-05-15T13:34:26.125'
end_time =  '2023-05-23'
tc.get_attitude('gaia', start_time, end_time, ts)
tc.get_state_vector('gaia', start_time, end_time, ts)
# tc.get_attitude('bc')

Function __get_url Took 24.4484 seconds
Function __get_url Took 12.5278 seconds


In [68]:
url = 'queryType=run&ops=ops_Routine&job=g_job_tmpl_fdtool_crttab_state.dat&opsConfig=opsSelect&jobConfig=jobTable&TIME_SCALE=TDB&INITIAL_TIME=2023%2F06%2F10+00%3A00%3A00.000&FINAL_TIME=2023%2F06%2F11+00%3A00%3A00.000&TIME_STEP=000+01%3A00%3A00.000&OBJECT=Gaia&REF_OBJECT=Solar+System+Barycentre&FRAME=mean+equatorial+J2000&LTC_FLAG=NO&scenarioId=NEW_ID&inputCase=DEFAULT_case'

parse_qs(url)

{'queryType': ['run'],
 'ops': ['ops_Routine'],
 'job': ['g_job_tmpl_fdtool_crttab_state.dat'],
 'opsConfig': ['opsSelect'],
 'jobConfig': ['jobTable'],
 'TIME_SCALE': ['TDB'],
 'INITIAL_TIME': ['2023/06/10 00:00:00.000'],
 'FINAL_TIME': ['2023/06/11 00:00:00.000'],
 'TIME_STEP': ['000 01:00:00.000'],
 'OBJECT': ['Gaia'],
 'REF_OBJECT': ['Solar System Barycentre'],
 'FRAME': ['mean equatorial J2000'],
 'LTC_FLAG': ['NO'],
 'scenarioId': ['NEW_ID'],
 'inputCase': ['DEFAULT_case']}